In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
df = pd.read_csv('egzoplanety_pl.csv')

In [85]:
print(df.head())

  Nazwa_Planety Gospodarz_Planety  Metoda_Odkrycia  \
0      11 Com b            11 Com  Radial Velocity   
1      11 UMi b            11 UMi  Radial Velocity   
2      14 And b            14 And  Radial Velocity   
3      14 Her b            14 Her  Radial Velocity   
4    16 Cyg B b          16 Cyg B  Radial Velocity   

                   Instytucja_Odkrywajaca Typ_Spektralny  \
0                        Xinglong Station         G8 III   
1  Thueringer Landessternwarte Tautenburg            NaN   
2       Okayama Astrophysical Observatory         K0 III   
3                  W. M. Keck Observatory            NaN   
4                  Multiple Observatories            NaN   

  Stosunek_Metalicznosci_Gwiazdy  Nr  Liczba_Gwiazd  Liczba_Planet  \
0                         [Fe/H]   1              2              1   
1                         [Fe/H]   2              1              1   
2                         [Fe/H]   3              1              1   
3                         [Fe/H]  

In [86]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4575 entries, 0 to 4574
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Nazwa_Planety                      4575 non-null   object 
 1   Gospodarz_Planety                  4575 non-null   object 
 2   Metoda_Odkrycia                    4575 non-null   object 
 3   Instytucja_Odkrywajaca             4575 non-null   object 
 4   Typ_Spektralny                     928 non-null    object 
 5   Stosunek_Metalicznosci_Gwiazdy     3192 non-null   object 
 6   Nr                                 4575 non-null   int64  
 7   Liczba_Gwiazd                      4575 non-null   int64  
 8   Liczba_Planet                      4575 non-null   int64  
 9   Rok_Odkrycia                       4575 non-null   int64  
 10  Okres_Orbitalny                    4413 non-null   float64
 11  Pol_Os_Orbity                      2763 non-null   float

In [87]:
print(df.isnull().sum())

Nazwa_Planety                           0
Gospodarz_Planety                       0
Metoda_Odkrycia                         0
Instytucja_Odkrywajaca                  0
Typ_Spektralny                       3647
Stosunek_Metalicznosci_Gwiazdy       1383
Nr                                      0
Liczba_Gwiazd                           0
Liczba_Planet                           0
Rok_Odkrycia                            0
Okres_Orbitalny                       162
Pol_Os_Orbity                        1812
Masa                                 2569
Mimosrod                             2868
Strumien_Insolacji                   4205
Temperatura_Rownowagi                3650
Efektywna_Temperatura_Gwiazdy         349
Promien_Gwiazdy                       447
Masa_Gwiazdy                          731
Metalicznosc_Gwiazdy                 1371
Grawitacja_Powierzchniowa_Gwiazdy     603
Odleglosc                             104
Wielkosc_Magnitudy                    175
dtype: int64


Przetwarzanie danych

In [88]:
df = df.drop(columns=['Nr', 'Nazwa_Planety', 'Gospodarz_Planety', 'Instytucja_Odkrywajaca', 'Typ_Spektralny', 'Stosunek_Metalicznosci_Gwiazdy'])
df = df.dropna().reset_index(drop=True)

In [89]:
categorical_cols = ['Metoda_Odkrycia']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

In [90]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Metoda_Odkrycia' in numeric_cols:
    numeric_cols.remove('Metoda_Odkrycia')
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

Model klasyfikacji przewidywania 'Metoda_Odkrycia'

In [91]:
X = df.drop('Metoda_Odkrycia', axis=1)
y = df['Metoda_Odkrycia']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność modelu klasyfikacyjnego: {accuracy:.2f}")

Dokładność modelu klasyfikacyjnego: 0.97


Model regresji przewidywania 'Temperatura_Rownowagi'

In [92]:
df_reg = df[df['Temperatura_Rownowagi'].notnull()].reset_index(drop=True)
X_reg = df_reg.drop('Temperatura_Rownowagi', axis=1)
y_reg = df_reg['Temperatura_Rownowagi']
numeric_cols_reg = X_reg.select_dtypes(include=[np.number]).columns.tolist()
if 'Metoda_Odkrycia' in numeric_cols_reg:
    numeric_cols_reg.remove('Metoda_Odkrycia')
X_reg[numeric_cols_reg] = scaler.fit_transform(X_reg[numeric_cols_reg])
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.20, random_state=42)
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X_train_reg, y_train_reg)
y_pred_reg = reg.predict(X_test_reg)
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
print(f"RMSE modelu regresyjnego: {rmse:.2f}")

RMSE modelu regresyjnego: 0.10


Przetwarzanie danych

In [93]:
df = pd.read_csv('egzoplanety_pl.csv')
cols_to_convert = [
    'Liczba_Gwiazd', 'Liczba_Planet', 'Rok_Odkrycia',
    'Okres_Orbitalny', 'Pol_Os_Orbity', 'Masa', 'Mimosrod',
    'Strumien_Insolacji', 'Temperatura_Rownowagi', 'Efektywna_Temperatura_Gwiazdy',
    'Promien_Gwiazdy', 'Masa_Gwiazdy', 'Metalicznosc_Gwiazdy',
    'Grawitacja_Powierzchniowa_Gwiazdy', 'Odleglosc', 'Wielkosc_Magnitudy'
]

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')
for col in cols_to_convert:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median())
categorical_cols = ['Metoda_Odkrycia', 'Typ_Spektralny']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

Model przewidywania okresu orbitalnego

In [94]:
from sklearn.metrics import r2_score

features = ['Pol_Os_Orbity', 'Masa', 'Mimosrod']
target = 'Okres_Orbitalny'
df_model1 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model1[features]
y = df_model1[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model1 = RandomForestRegressor(n_estimators=100, random_state=42)
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Okres_Orbitalny: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Okres_Orbitalny: {r2:.2f}")

RMSE dla modelu przewidującego Okres_Orbitalny: 0.00
R² dla modelu przewidującego Okres_Orbitalny: 0.84


Model przewidywania masy planety

In [95]:
features = ['Masa_Gwiazdy', 'Promien_Gwiazdy', 'Metalicznosc_Gwiazdy', 'Pol_Os_Orbity', 'Mimosrod']
target = 'Masa'
df_model2 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model2[features]
y = df_model2[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model2 = RandomForestRegressor(n_estimators=100, random_state=42)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Masę planety: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Masę planety: {r2:.2f}")

RMSE dla modelu przewidującego Masę planety: 0.62
R² dla modelu przewidującego Masę planety: 0.39


Model przewidywania ekscentryczności

In [96]:
features = ['Okres_Orbitalny', 'Pol_Os_Orbity', 'Masa_Gwiazdy', 'Masa']
target = 'Mimosrod'
df_model3 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model3[features]
y = df_model3[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model3 = RandomForestRegressor(n_estimators=100, random_state=42)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Mimosrod: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Masę planety: {r2:.2f}")

RMSE dla modelu przewidującego Mimosrod: 0.99
R² dla modelu przewidującego Masę planety: 0.14


Model przewidywania temperatury równowagi

In [97]:
if 'Typ_Spektralny' not in categorical_cols:
    df['Typ_Spektralny'] = df['Typ_Spektralny'].astype(str)
    le = LabelEncoder()
    df['Typ_Spektralny'] = le.fit_transform(df['Typ_Spektralny'])
    label_encoders['Typ_Spektralny'] = le
features = ['Strumien_Insolacji', 'Pol_Os_Orbity', 'Typ_Spektralny', 'Efektywna_Temperatura_Gwiazdy']
target = 'Temperatura_Rownowagi'
df_model4 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model4[features]
y = df_model4[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model4 = RandomForestRegressor(n_estimators=100, random_state=42)
model4.fit(X_train, y_train)
y_pred = model4.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Temperaturę Równowagi: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Masę planety: {r2:.2f}")

RMSE dla modelu przewidującego Temperaturę Równowagi: 0.06
R² dla modelu przewidującego Masę planety: 0.99


Model przewidywania strumienia promieniowania

In [98]:
features = ['Typ_Spektralny', 'Efektywna_Temperatura_Gwiazdy', 'Pol_Os_Orbity']
target = 'Strumien_Insolacji'
df_model5 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model5[features]
y = df_model5[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model5 = RandomForestRegressor(n_estimators=100, random_state=42)
model5.fit(X_train, y_train)
y_pred = model5.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Strumień Insolacji: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Masę planety: {r2:.2f}")

RMSE dla modelu przewidującego Strumień Insolacji: 0.26
R² dla modelu przewidującego Masę planety: 0.05


Model przewidywania parametrow gwiazdy (metaliczność)

In [99]:
features = ['Typ_Spektralny', 'Efektywna_Temperatura_Gwiazdy', 'Masa_Gwiazdy', 'Promien_Gwiazdy', 'Okres_Orbitalny', 'Pol_Os_Orbity']
target = 'Metalicznosc_Gwiazdy'
df_model6 = df[features + [target]].dropna().reset_index(drop=True)
X = df_model6[features]
y = df_model6[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model6 = RandomForestRegressor(n_estimators=100, random_state=42)
model6.fit(X_train, y_train)
y_pred = model6.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE dla modelu przewidującego Metaliczność Gwiazdy: {rmse:.2f}")
r2 = r2_score(y_test, y_pred)
print(f"R² dla modelu przewidującego Masę planety: {r2:.2f}")

RMSE dla modelu przewidującego Metaliczność Gwiazdy: 0.91
R² dla modelu przewidującego Masę planety: 0.37
